In [46]:
#@title Step 1: Install lm-evaluation-harness
# Install the library
# !pip install git+https://github.com/EleutherAI/lm-evaluation-harness.git

# Install necessary dependencies for loading datasets and specific models if needed
# !pip install datasets evaluate accelerate transformers torch einops rouge-score sacrebleu sentencepiece

In [51]:
# Huggingface token if needed
from huggingface_hub import login
login()

In [52]:
#@title Step 2: Split Data into Train and Test Sets

import json
import random

# --- Configuration ---
input_json_file = 'norwegian_medical_qa.json' # Make sure this is the correct name of your uploaded file
train_output_file = 'train_split.json'
test_output_file = 'test_split.json'
test_split_ratio = 0.20 # Use 20% of the data for testing
random_seed = 42 # for reproducible shuffling
# -------------------

random.seed(random_seed) # Set seed for shuffling

try:
    # Load the original data
    with open(input_json_file, 'r', encoding='utf-8') as f:
        all_data = json.load(f)

    # Shuffle the data
    random.shuffle(all_data)

    # Determine split index
    num_total = len(all_data)
    num_test = int(num_total * test_split_ratio)
    num_train = num_total - num_test

    # Split the data
    train_data = all_data[:num_train]
    test_data = all_data[num_train:]

    # Save the training split
    with open(train_output_file, 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(train_data)} entries to {train_output_file}")

    # Save the test split
    with open(test_output_file, 'w', encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent=2)
    print(f"Saved {len(test_data)} entries to {test_output_file}")

except FileNotFoundError:
    print(f"ERROR: Input file not found: {input_json_file}")
    print("Please make sure you have uploaded the correct file in Step 2.")
except Exception as e:
    print(f"An error occurred during splitting: {e}")

Saved 993 entries to train_split.json
Saved 248 entries to test_split.json


In [53]:
#@title Step 3: Create Directories
import os

# Define the path for where the task is
task_dir = "/content/lm_eval/tasks/norwegian_medical_qa"

# Create the directories if they do not exist
os.makedirs(task_dir, exist_ok=True)

print(f"Directory structure created at: {task_dir}")

Directory structure created at: /content/lm_eval/tasks/norwegian_medical_qa


In [54]:
#@title Step 4: Few-Shot Prompting (2-shot)
%%writefile /content/lm_eval/tasks/norwegian_medical_qa/norwegian_medical_qa.yaml
# Task identifier
task: norwegian_medical_qa
task_alias: Norwegian Medical Q&A
output_type: generate_until

# Dataset Configuration
dataset_path: json
dataset_name: null
dataset_kwargs:
  #data_files: /content/norwegian_medical_qa.json
  data_files:
    train: /content/train_split.json
    test: /content/test_split.json

# Use splits created before
test_split: test
fewshot_split: train

# Prompt and target
doc_to_text: "Spørsmål: {{question_text}}\nSvar:"
doc_to_target: answer_text

# --- Choose num_fewshot ---
num_fewshot: 2

# --- Metric list ---
metric_list:
   - metric: exact_match
     aggregation: mean
     higher_is_better: true
   - metric: rouge
     aggregation: mean
     higher_is_better: true

generation_kwargs:
  max_gen_toks: 256
  until: ["<eos>", "\n\n"] # Stop at EOS or double newline

metadata:
  version: 0.0
  description: "Benchmark based on a custom dataset of Norwegian medical questions and answers."

Overwriting /content/lm_eval/tasks/norwegian_medical_qa/norwegian_medical_qa.yaml


In [56]:
#@title Step 5j: Run Evaluation with Gemma (Predict Only - Few-Shot), use #    --limit 10 \ if you want to test on smaller set of data
!rm -rf ./predict_results_gemma_fewshot # Clean previous results if any
!lm_eval \
    --model hf \
    --model_args pretrained="google/gemma-3-1b-it" \
    --tasks norwegian_medical_qa \
    --batch_size auto \
    --device cuda \
    --include_path /content/lm_eval/tasks/ \
    --log_samples \
    --output_path ./predict_results_gemma_fewshot

2025-05-01 19:02:39.828920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746126159.857632   21655 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746126159.867785   21655 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-01:19:02:46 INFO     [__main__:347] Including path: /content/lm_eval/tasks/
2025-05-01:19:02:51 INFO     [__main__:440] Selected Tasks: ['norwegian_medical_qa']
2025-05-01:19:02:51 INFO     [evaluator:185] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-05-01:19:02:51 INFO     [evaluator:223] Initializing hf model, with arguments: {'pretraine